In [1]:
import numpy as  np
import os
import copy
import open3d as o3d
from scipy.spatial.transform import Rotation as R

from sys import argv, exit

np.random.seed(42)
np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})

In [2]:
def frobNorm(P1, P2, str1="mat1", str2="mat2"):
    np.set_printoptions(suppress=True)
    val = np.linalg.norm(P1 - P2, 'fro')
    print(f"Frobenius norm between {str1} and {str2} is: {val}")
    return val

In [3]:
# Have 2/3/4 points

p = np.array([[0,9,0],[1,1,1],[1,2,3],[7,8,13]]) # You can add more points if you want to.
p_homo = np.hstack((p, np.ones(p.shape[0]).reshape((p.shape[0],1))))

T = np.identity(4)
T[0:3, 0:3] = R.from_euler('zyx', np.random.rand(3) * 90, degrees = True).as_matrix() #change to [30/90,45,60]
# T[0:3, 0:3] = R.from_euler('zyx', [90, 45, 30], degrees = True).as_matrix() #change to [30/90,45,60]
# T[0:3, 0:3] = R.from_euler('z', [45], degrees = True).as_matrix() #60/30
T[0,3], T[1,3], T[2,3] = np.random.rand(3) * 10

In [4]:
q_homo_T = np.matmul(T, p_homo.T)
q_homo = q_homo_T.T
q = np.delete(q_homo, -1, 1)

In [5]:
def icp(p, q, min_points, method="Wahba"): 
    # "OP" stands for Orthogonal Procrustes which needs minimum 4 points
    # "Wahba" only needs 3 points
    
    # Sidenote:
    # if you are just putting 2 inside the for loop below in case of min_points =2 but using all 3 point info
    # for steps like calculating mean, 
    # that is not correct since you are already using 3 points information for calculating centroid.
    # In essence, you are using 3 points even if iterating for 2 points here
    # since p_dash/q_dash is a vector joining centroid and point. Put geometry on paper now for further understanding.
    min_points = min_points
    print(f"min_points={min_points}")
    u_p = np.mean(p[:min_points], axis=0)
    u_q = np.mean(q[:min_points], axis=0)    
    p_dash = p[:min_points] - u_p
    q_dash = q[:min_points] - u_q
    W = np.zeros((3,3))
    for i in range(min_points):
        W += np.matmul(q_dash[i, np.newaxis].T, p_dash[i, np.newaxis])
    u, s, vh = np.linalg.svd(W, full_matrices=True)
    print(f"Singular values are {str(s)}")
    M = np.diag([1,1, np.linalg.det(u) * np.linalg.det(vh)])
    if method == "Wahba":
        R_recovered = u @ M @ vh
    elif method == "OP":
        R_recovered = u @ vh
    # actually above is correct only, but only works if rank(W) > 3. If =2, then you need to use M.
    t_recovered = u_q - R_recovered @ u_p
    T_recovered = np.hstack((R_recovered, np.array([[t_recovered[0]], [t_recovered[1]], [t_recovered[2]]])))
    T_recovered = np.vstack((T_recovered, np.array([[0,0,0,1]])))
        
    

    return T_recovered

### Set method: Orthogonal Procrustes or Wahba?

* "OP" stands for Orthogonal Procrustes which needs minimum 4 points
* "Wahba" only needs 3 points

In [6]:
method = "OP" # "Wahba" or "OP"
if method == "OP":
    method_full = "Orthogonal Procrustes"

In [7]:
min_points = 2
T_recovered = icp(p, q, min_points, method=method)
recovered_bool = float(frobNorm(T, T_recovered, str1="original T", str2="Recovered T")) < 1e-5
print(f"\n {method_full} VERDICT: Using {min_points} points, it is {recovered_bool} that I recovered the original transform \n \n")



min_points = 3
T_recovered = icp(p, q, min_points, method=method)
recovered_bool = float(frobNorm(T, T_recovered, str1="original T", str2="Recovered T")) < 1e-5
print(f"\n {method_full} VERDICT: Using {min_points} points, it is {recovered_bool} that I recovered the original transform \n \n")



min_points = 4
T_recovered = icp(p, q, min_points, method=method)
recovered_bool = float(frobNorm(T, T_recovered, str1="original T", str2="Recovered T")) < 1e-5
print(f"\n {method_full} VERDICT: Using {min_points} points, it is {recovered_bool} that I recovered the original transform \n \n")

min_points=2
Singular values are [33.00000 0.00000 0.00000]
Frobenius norm between original T and Recovered T is: 2.8372086335363798

 Orthogonal Procrustes VERDICT: Using 2 points, it is False that I recovered the original transform 
 

min_points=3
Singular values are [40.9395607   2.39377263  0.        ]
Frobenius norm between original T and Recovered T is: 2.8996833043120636

 Orthogonal Procrustes VERDICT: Using 3 points, it is False that I recovered the original transform 
 

min_points=4
Singular values are [146.11953215  40.92832629   0.45214156]
Frobenius norm between original T and Recovered T is: 4.1514596760159754e-15

 Orthogonal Procrustes VERDICT: Using 4 points, it is True that I recovered the original transform 
 



In [8]:
method = "Wahba" # "Wahba" or "OP"

In [9]:
min_points = 2
T_recovered = icp(p, q, min_points, method=method)
recovered_bool = float(frobNorm(T, T_recovered, str1="original T", str2="Recovered T")) < 1e-5
print(f"\n {method} VERDICT: Using {min_points} points, it is {recovered_bool} that I recovered the original transform \n \n")



min_points = 3
T_recovered = icp(p, q, min_points, method=method)
recovered_bool = float(frobNorm(T, T_recovered, str1="original T", str2="Recovered T")) < 1e-5
print(f"\n {method} VERDICT: Using {min_points} points, it is {recovered_bool} that I recovered the original transform \n \n")



min_points = 4
T_recovered = icp(p, q, min_points, method=method)
recovered_bool = float(frobNorm(T, T_recovered, str1="original T", str2="Recovered T")) < 1e-5
print(f"\n {method} VERDICT: Using {min_points} points, it is {recovered_bool} that I recovered the original transform \n \n")

min_points=2
Singular values are [33.  0.  0.]
Frobenius norm between original T and Recovered T is: 2.8372086335363798

 Wahba VERDICT: Using 2 points, it is False that I recovered the original transform 
 

min_points=3
Singular values are [40.9395607   2.39377263  0.        ]
Frobenius norm between original T and Recovered T is: 4.7775582088979414e-15

 Wahba VERDICT: Using 3 points, it is True that I recovered the original transform 
 

min_points=4
Singular values are [146.11953215  40.92832629   0.45214156]
Frobenius norm between original T and Recovered T is: 4.328229281970082e-15

 Wahba VERDICT: Using 4 points, it is True that I recovered the original transform 
 

